In [46]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from google.colab import drive
from zipfile import ZipFile

In [ ]:
# Utilizing gpu device for fast and efficient code run
device_list = tf.test.gpu_device_name()
device_list

'/device:GPU:0'

In [ ]:
if device_list != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_list))

Found GPU at: /device:GPU:0


In [ ]:
# Path to needed csv files
training_csv = '/content/drive/MyDrive/Planet/train_v2.csv'
testing_csv = '/content/drive/MyDrive/Planet/sample_submission_v2.csv'

In [ ]:
# Unzipping the large jpg files
with ZipFile('/content/drive/MyDrive/Planet/train-jpg.zip', 'r') as zipObj:
  zipObj.extractall('sample_data/new_train')

with ZipFile('/content/drive/MyDrive/Planet/test-jpg.zip', 'r') as zipObj:
  zipObj.extractall('sample_data/new_test')

with ZipFile('/content/drive/MyDrive/Planet/test-jpg-additional.zip', 'r') as zipObj:
  zipObj.extractall('sample_data/new_add_test')

In [ ]:
# Reading in the training data
training_class = pd.read_csv(training_csv)
training_class.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [47]:
# Creating a function to split and store unique tags
labels = set()
def splitting_tags(tags):
    for tag in tags.split():
        labels.add(tag)

# Creating a copy of the training class dataset to work on
training_class_dummy = training_class.copy()
training_class_dummy['tags'].apply(splitting_tags)
labels = list(labels)
print(labels)

['primary', 'clear', 'cultivation', 'cloudy', 'haze', 'blow_down', 'habitation', 'selective_logging', 'conventional_mine', 'slash_burn', 'bare_ground', 'blooming', 'agriculture', 'artisinal_mine', 'partly_cloudy', 'water', 'road']


In [48]:
#Confirming  that the length of the df is the same as the shape
assert len(training_class_dummy['image_name'].unique()) == training_class_dummy.shape[0]

In [ ]:
##One hot encoding is performed on the labels in train classes
for tag in labels:
    training_class_dummy[tag] = training_class_dummy['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
## adding .jpg extension to the column image_name so as to have same name format as the image files
training_class_dummy['image_name'] = training_class_dummy['image_name'].apply(lambda x: '{}.jpg'.format(x))
training_class_dummy.head()

,image_name,tags,primary,clear,cultivation,cloudy,haze,blow_down,habitation,selective_logging,conventional_mine,slash_burn,bare_ground,blooming,agriculture,artisinal_mine,partly_cloudy,water,road
0,train_0.jpg,haze primary,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1.jpg,agriculture clear primary water,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,train_2.jpg,clear primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3.jpg,clear primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1


In [ ]:
#importing necessary libraries for training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Checking out our newly created features via one hot encoding. Actually started from index 2
columns = list(training_class_dummy.columns[2:]) 
columns

['primary',
 'clear',
 'cultivation',
 'cloudy',
 'haze',
 'blow_down',
 'habitation',
 'selective_logging',
 'conventional_mine',
 'slash_burn',
 'bare_ground',
 'blooming',
 'agriculture',
 'artisinal_mine',
 'partly_cloudy',
 'water',
 'road']

In [49]:
#define a function for fbeta scoring
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):
    
    beta_squared = beta**2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb

In [ ]:
#define a function for accuracy for multi_label classification
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
        
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32) * tf.cast(tf.logical_not(y_pred), tf.float32), 
                       axis = 1)
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [50]:
#defining our model using a function build_model()
def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    opt = Adam(learning_rate=1e-4)

    model.compile(loss='binary_crossentropy',
              # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=opt,
              metrics=[multi_label_acc, fbeta])

    return model

In [ ]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_fbeta', mode = 'max',
                                       save_best_only = True, save_weights_only = True)

In [ ]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

#generating train data generator which is 80% of the train dataset
#note that a generator contains both features and target of the data
train_generator = train_image_gen.flow_from_dataframe(dataframe=training_class_dummy,
                                                directory ="/content/sample_data/new_train/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="training", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=training_class_dummy,
                                                directory ="/content/sample_data/new_train/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="validation", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

Found 32384 validated image filenames.
Found 8095 validated image filenames.


In [ ]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

In [51]:
#initializing the model
model1 = build_model()

In [ ]:
#this shows the summary of the model, simply put as the model architecture
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 128, 128, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        1

In [ ]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, steps_per_epoch = step_train_size, validation_data = val_generator, 
           validation_steps = step_val_size,epochs = 5, 
           callbacks = [save_best_check_point])

Epoch 1/5
2024/2024 [==============================] - 132s 60ms/step - loss: 0.2172 - multi_label_acc: 0.9172 - fbeta: 0.7028 - val_loss: 0.1679 - val_multi_label_acc: 0.9365 - val_fbeta: 0.7750
Epoch 2/5
2024/2024 [==============================] - 115s 57ms/step - loss: 0.1614 - multi_label_acc: 0.9367 - fbeta: 0.7903 - val_loss: 0.1373 - val_multi_label_acc: 0.9451 - val_fbeta: 0.8138
Epoch 3/5
2024/2024 [==============================] - 114s 56ms/step - loss: 0.1447 - multi_label_acc: 0.9427 - fbeta: 0.8147 - val_loss: 0.1304 - val_multi_label_acc: 0.9485 - val_fbeta: 0.8302
Epoch 4/5
2024/2024 [==============================] - 115s 57ms/step - loss: 0.1353 - multi_label_acc: 0.9466 - fbeta: 0.8292 - val_loss: 0.1271 - val_multi_label_acc: 0.9492 - val_fbeta: 0.8289
Epoch 5/5
2024/2024 [==============================] - 117s 58ms/step - loss: 0.1283 - multi_label_acc: 0.9492 - fbeta: 0.8396 - val_loss: 0.1183 - val_multi_label_acc: 0.9540 - val_fbeta: 0.8627


In [52]:
#initializing a second model so we can make predictions
model2 = build_model()

In [ ]:
#adding .jpg extension to image name in the sample submission file
sample_submission = pd.read_csv('/content/drive/MyDrive/Planet/sample_submission_v2.csv')
sample_submission1 = sample_submission.copy()
sample_submission1['image_name'] = sample_submission1['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_submission1.head()

,image_name,tags
0,test_0.jpg,primary clear agriculture road water
1,test_1.jpg,primary clear agriculture road water
2,test_2.jpg,primary clear agriculture road water
3,test_3.jpg,primary clear agriculture road water
4,test_4.jpg,primary clear agriculture road water


In [ ]:
#loading in the weights of the trained model so we can make predictions with it
model2.load_weights('best_model.hdf5')

In [ ]:
# we divide the sample submission file into two splits, first test1_df which contains the first 40669 images 
test1_df = sample_submission1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test1_df.head()

,image_name
0,test_0.jpg
1,test_1.jpg
2,test_2.jpg
3,test_3.jpg
4,test_4.jpg


In [ ]:
#initializing imagedatagenerator for the test images and also rescaling
test_image_gen = ImageDataGenerator(rescale = 1/255)


#creating a generator for the images found in the first test image files
test_generator1 = test_image_gen.flow_from_dataframe(dataframe=test1_df, 
                                                directory="/content/sample_data/new_test/test-jpg", 
                                                x_col="image_name", y_col=None, batch_size=16, 
                                                shuffle=False, class_mode=None, target_size=(128,128))

step_test_size1 = int(np.ceil(test_generator1.samples/test_generator1.batch_size))

Found 40669 validated image filenames.


In [ ]:
#first, we reset the test generator to avoid shuffling of index as we want it to be orderly
test_generator1.reset()
pred1 = model2.predict(test_generator1, steps = step_test_size1, verbose = 1)

2542/2542 [==============================] - 103s 40ms/step


In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
file_names1 = test_generator1.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5 
pred_tags1 = pd.DataFrame(pred1)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this 
result1 = pd.DataFrame({'image_name': file_names1, 'tags': pred_tags1})
result1.head()

,image_name,tags
0,test_0.jpg,primary clear
1,test_1.jpg,primary clear
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,primary clear
4,test_4.jpg,primary partly_cloudy


In [ ]:
#second batch of the test dataset
test2_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
test2_df.head()

,image_name
0,file_0.jpg
1,file_1.jpg
2,file_10.jpg
3,file_100.jpg
4,file_1000.jpg


In [ ]:
#creating a generator for the second batch of test image files
test_generator2 = test_image_gen.flow_from_dataframe(dataframe=test2_df, 
                                                directory="/content/sample_data/new_add_test/test-jpg-additional", 
                                                x_col="image_name", y_col=None, batch_size=16, 
                                                shuffle=False, class_mode=None, target_size=(128,128))

step_test_size2 = int(np.ceil(test_generator2.samples/test_generator2.batch_size))

Found 20522 validated image filenames.


In [ ]:
#we reset the generator to avoid shuffling, then make prediction on the generator
test_generator2.reset()
pred2 = model2.predict(test_generator2, steps = step_test_size2, verbose = 1)

1283/1283 [==============================] - 51s 40ms/step


In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
file_names2 = test_generator2.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5
pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ''.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': file_names2, 'tags': pred_tags2})
result2.head()

,image_name,tags
0,file_0.jpg,primaryclear
1,file_1.jpg,primaryagriculturepartly_cloudyroad
2,file_10.jpg,primaryagriculturewaterroad
3,file_100.jpg,primaryclearagriculture
4,file_1000.jpg,primaryclear


In [ ]:
#for the final result of the predicted tags for the test images, we need to concat the first and second results in 
#that order to avoid shuffling the index
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
#print the final result
last_result.head()

(61191, 2)


,image_name,tags
0,test_0.jpg,primary clear
1,test_1.jpg,primary clear
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,primary clear
4,test_4.jpg,primary partly_cloudy


In [ ]:
#Removing the .jpg extension from the image_name of the last_result which was added for ease of manipulation of the data.
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary clear
4,test_4,primary partly_cloudy


In [53]:
# #Finally, we save the result to a csv file using the .to_csv() method and setting the index to false.
last_result.to_csv('submission.csv', index = False)